In [1]:
import torch.nn as nn
import torch

n_points = 100

model = nn.Sequential(
    nn.Linear(in_features = n_points, out_features= 256),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(128, 128),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(128, n_points),
)

In [3]:
import numpy as np
V = np.loadtxt('../in/V.txt', dtype=float)
waves = np.loadtxt('../in/waves.txt', dtype=float)

In [4]:
from sklearn.model_selection import train_test_split

# Split train and test 
idx_train, idx_test, wave_train, wave_test = train_test_split(np.arange(V.shape[0]),
                                                              waves, test_size=0.33, random_state=123)
V_train = V[idx_train,:]
V_test = V[idx_test,:]

In [14]:
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

In [3]:
n_points = 100

epochs = 10000


for i in range(epochs):

  y = model(torch.from_numpy(V_train).float())

  optimizer.zero_grad()

  losss = loss(y, torch.from_numpy(wave_train).float())

  losss.backward()

  optimizer.step()

  if i%100 == 0:
    print(f'Epoch {i}/{epochs} Loss {losss}')

In [5]:
model = torch.load('../Models/model_adam.pt')
model.eval()

Sequential(
  (0): Linear(in_features=100, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.1, inplace=False)
  (3): Linear(in_features=256, out_features=256, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.1, inplace=False)
  (6): Linear(in_features=256, out_features=128, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.1, inplace=False)
  (9): Linear(in_features=128, out_features=128, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.1, inplace=False)
  (12): Linear(in_features=128, out_features=100, bias=True)
)

In [8]:
predict = model(torch.from_numpy(V_test).float())

print("Test MSE: %f" % loss(predict, torch.from_numpy(wave_test).float()))
predict = predict.detach().numpy()

Test MSE: 0.000031


In [11]:
xmin = -8
xmax = 8

x = np.arange(xmin, xmax, (xmax - xmin)/n_points)

In [13]:
import plotly
from plotly.graph_objs import graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

idx = 2
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=x,
    y=predict[idx,:],
    mode="lines",
    name = "prediction phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=wave_test[idx,:],
    mode="lines",
    name = "real phi"),
    secondary_y=False
    )

fig.add_trace(go.Scatter(
    x=x,
    y=V_test[idx,:],
    mode="lines",
    name = "Potential"),
    secondary_y=True
    )

fig.update_layout(
    title="Random potential prediction",
    xaxis_title="x")

fig.show()